In [37]:
import os
import sys
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [3]:
import figure_setup as fs

# Get ZTF fit

In [16]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")
damit_file = os.path.join(data_fink, "data", "damit_abc.csv")

In [17]:
# Threshold for selection (of non-zero values)
thres = 1e-3

# Minimum phase angle to consider
min_phase = 3

In [18]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

fink_colors = ["#15284F", "#F5622E"]

color_fink = "#15284F"

In [19]:
damit = pd.read_csv(damit_file)
# damit.head(5)

In [20]:
damit

,model_id,asteroid_id,lambda,beta,period,number,name,designation,reference_id,bibcode,a,b,c,R
0,101,101,35.0,-12.0,7.813230,2.0,Pallas,NaN,106,2003icar..164..346t,2.600240e+06,2.464155e+06,2.282461e+06,0.902027
1,101,101,35.0,-12.0,7.813230,2.0,Pallas,NaN,139,2011icar..214..652d,2.600240e+06,2.464155e+06,2.282461e+06,0.902027
2,106,104,340.0,42.0,7.274471,6.0,Hebe,NaN,106,2003icar..164..346t,1.690630e+05,1.634407e+05,1.447296e+05,0.870793
3,106,104,340.0,42.0,7.274471,6.0,Hebe,NaN,139,2011icar..214..652d,1.690630e+05,1.634407e+05,1.447296e+05,0.870793
4,110,106,335.0,-5.0,12.866670,8.0,Flora,NaN,106,2003icar..164..346t,8.951272e+04,8.496190e+04,7.833080e+04,0.898516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16312,16275,10853,326.0,-83.0,8.016800,99949.0,Miepgies,NaN,665,2023arxiv230510798d,8.597170e-01,5.586180e-01,5.387130e-01,0.795492
16313,16276,10854,34.0,-42.0,143.500000,99973.0,NaN,NaN,665,2023arxiv230510798d,9.555900e-01,5.506200e-01,4.848120e-01,0.693913
16314,16277,10854,194.0,-64.0,143.490000,99973.0,NaN,NaN,665,2023arxiv230510798d,1.003388e+00,5.521440e-01,4.607300e-01,0.646806
16315,16278,10855,192.0,-52.0,7.071400,99990.0,NaN,NaN,665,2023arxiv230510798d,1.090510e+00,5.518300e-01,4.180810e-01,0.570504


In [21]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [22]:
data.columns

Index(['ssnamenr', 'HG_chi2red', 'HG_status', 'HG_fit', 'HG_rms', 'HG_rms_g',
       'HG_rms_r', 'HG_median_error_phot', 'HG_median_error_phot_1',
       'HG_median_error_phot_2', 'HG_H_g', 'HG_dH_g', 'HG_G_g', 'HG_dG_g',
       'HG_H_r', 'HG_dH_r', 'HG_G_r', 'HG_dG_r', 'HG_flag', 'HG1G2_chi2red',
       'HG1G2_status', 'HG1G2_fit', 'HG1G2_rms', 'HG1G2_rms_g', 'HG1G2_rms_r',
       'HG1G2_median_error_phot', 'HG1G2_median_error_phot_1',
       'HG1G2_median_error_phot_2', 'HG1G2_H_g', 'HG1G2_dH_g', 'HG1G2_G1_g',
       'HG1G2_dG1_g', 'HG1G2_G2_g', 'HG1G2_dG2_g', 'HG1G2_H_r', 'HG1G2_dH_r',
       'HG1G2_G1_r', 'HG1G2_dG1_r', 'HG1G2_G2_r', 'HG1G2_dG2_r', 'HG1G2_flag',
       'SHG1G2_chi2red', 'SHG1G2_min_cos_lambda', 'SHG1G2_mean_cos_lambda',
       'SHG1G2_max_cos_lambda', 'SHG1G2_status', 'SHG1G2_fit', 'SHG1G2_rms',
       'SHG1G2_rms_g', 'SHG1G2_rms_r', 'SHG1G2_median_error_phot',
       'SHG1G2_median_error_phot_1', 'SHG1G2_median_error_phot_2', 'n_obs',
       'n_obs_g', 'n_obs_r', 

In [23]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [62]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



# Histogram of oblateness R

In [66]:
maskFINK = maskFINK_union

# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.98,
        "bottom": 0.13,
        "left": 0.10,
        "right": 0.98,
    },
)

# --------------------------------------------------------------------------------
# FINK R
r = [0.31, 1]
b = 50


ax.hist(
    damit["R"],
    range=r,
    bins=b,
    density=True,
    label=f"DAMIT ({len(damit):,d})",
#     color="slategray",
    lw=1,
    ls='--',
    histtype='step',
    alpha=0.8,
    rasterized=True,
)


ax.hist(
    data.loc[maskFINK, "SHG1G2_R"],
    range=r,
    bins=b,
    density=True,
    label=f"FINK ({len(data[maskFINK]):,d})",
#     color=color_fink,
    lw=2,
    histtype='step',
    alpha=0.8,
    rasterized=True,
)


# --------------------------------------------------------------------------------
# Axes
ax.set_xlabel("Oblateness R")
ax.set_ylabel("Density count")

ax.set_ylim(0, 5)
ax.legend(loc="upper left")

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "distrib_R.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "distrib_R.pgf"),
)
plt.close()

In [67]:
print(f"\\newcommand{{\damitSSO}}{{{damit['asteroid_id'].nunique():,d}\\xspace}}")
print(f"\\newcommand{{\damitModel}}{{{len(damit):,d}\\xspace}}")

\newcommand{\damitSSO}{10,743\xspace}
\newcommand{\damitModel}{16,317\xspace}


below is exploratory

# Check RMS of SSOs with shape models

In [68]:
nn = rocks.id(data.ssnamenr)
data["sso_name"] = [n[0] for n in nn]

ERROR    [rocks] Could not identify '2009 PA09'.

ERROR    [rocks] Could not identify '2016 XB07'.

ERROR    [rocks] Could not identify '2006 QB03'.

ERROR    [rocks] Could not identify '2013 WJ00'.

ERROR    [rocks] Could not identify '2020 OW03'.

...

ERROR    [rocks] Could not identify '2007 TG08'.

ERROR    [rocks] Could not identify '2020 OL03'.

ERROR    [rocks] Could not identify '2009 CD04'.

ERROR    [rocks] Could not identify '2012 UA00'.

ERROR    [rocks] Could not identify '2020 KJ06'.

In [69]:
nn = rocks.id(damit.number)
damit["sso_name"] = [n[0] for n in nn]

In [70]:
maskFINK = maskFINK_union

fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 0.5]
b = 50

# all = data[maskFINK].merge(damit, on="sso_name", how="left")


# Histogram FINK
ax.hist(
    data.loc[maskFINK, "SHG1G2_rms"],
    range=r,
    bins=b,
    color=color_fink,
    density=True,
    alpha=0.8,
    label=f"FINK ({len(data[maskFINK]):,d})",
)


# Histogram 3dShape
cond = maskFINK & data.sso_name.isin(damit.sso_name)
ax.hist(
    data.loc[cond, "SHG1G2_rms"],
    range=r,
    bins=b,
    color="slategray",
    density=True,
    alpha=0.8,
    label=f"With 3D ({len(data[cond]):,d})",
)

# Axes
ax.legend(loc="upper right")

fig.savefig(os.path.join("..", "gfx", "rms_shape.png"), facecolor="white", dpi=150)
fig.savefig(os.path.join("..", "gfx", "rms_shape.pgf"))

In [71]:
maskFINK = maskFINK_union

fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 15]
b = 50


# Histogram FINK
ax.hist(
    data.loc[maskFINK, "SHG1G2_chi2red"],
    range=r,
    bins=b,
    color=color_fink,
    density=True,
    alpha=0.8,
    label=f"FINK ({len(data[maskFINK]):,d})",
)


# Histogram 3dShape
cond = maskFINK & data.sso_name.isin(damit.sso_name)
ax.hist(
    data.loc[cond, "SHG1G2_chi2red"],
    range=r,
    bins=b,
    color="slategray",
    density=True,
    alpha=0.8,
    label=f"With 3D ({len(data[cond]):,d})",
)

# Axes
ax.legend(loc="upper right")

fig.savefig(os.path.join("..", "gfx", "chi2_shape.png"), facecolor="white", dpi=150)
fig.savefig(os.path.join("..", "gfx", "chi2_shape.pgf"))

In [72]:
s = 3000
data[cond].sort_values("SHG1G2_chi2red").reset_index().loc[
    s : s + 10,
    ["ssnamenr", "SHG1G2_R", "SHG1G2_obliquity", "SHG1G2_chi2red", "SHG1G2_rms"],
]

,ssnamenr,SHG1G2_R,SHG1G2_obliquity,SHG1G2_chi2red,SHG1G2_rms
3000,49820,0.402635,149.005098,5.584586,0.183478
3001,93239,0.748642,8.171180,5.586797,0.223177
3002,55946,0.628452,168.171621,5.587711,0.141211
3003,140437,0.732720,154.733855,5.592798,0.233650
3004,26128,0.615057,13.409706,5.593288,0.162524
3005,3827,0.943088,103.688345,5.593608,0.126558
3006,10226,0.697809,145.385949,5.594495,0.125789
3007,16270,0.484784,149.075460,5.595002,0.104835
3008,13460,0.920232,41.138961,5.595395,0.133526
3009,67333,0.320074,155.941659,5.595478,0.204004


Some LC signature I think
- 9241


In [59]:
maskFINK = maskFINK_union

all = data[maskFINK].merge(damit, on="sso_name", how="left")

fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

ax.scatter(all.a / all.b, all.SHG1G2_rms, s=1, alpha=0.5)
ax.set_xlabel("a/b")
ax.set_ylabel("RMS")

fig.savefig(os.path.join("..", "gfx", "rms_vs_ab.png"), facecolor="white", dpi=150)


fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

ax.scatter(all.R, all.SHG1G2_rms, s=1, alpha=0.5)
ax.set_xlabel("R")
ax.set_ylabel("RMS")

fig.savefig(os.path.join("..", "gfx", "rms_vs_R.png"), facecolor="white", dpi=150)

RuntimeError: The command
    xelatex -interaction=nonstopmode -halt-on-error figure.tex
failed and generated the following output:
This is XeTeX, Version 3.141592653-2.6-0.999993 (TeX Live 2022/dev/Debian) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./figure.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size12.clo))
(/usr/share/texlive/texmf-dist/tex/latex/hyperref/hyperref.sty
(/usr/share/texlive/texmf-dist/tex/generic/ltxcmds/ltxcmds.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)
(/usr/share/texlive/texmf-dist/tex/generic/pdftexcmds/pdftexcmds.sty
(/usr/share/texlive/texmf-dist/tex/generic/infwarerr/infwarerr.sty))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/kvsetkeys/kv

# Test scatter DAMIT.R vs FINK.R

In [47]:
maskFINK = maskFINK_union

fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 1]
b = 50

all = data[maskFINK].merge(damit, on="sso_name", how="left")

# Histogram
ax.scatter(
    all["SHG1G2_R"],
    all.R,
    s=1,
    marker=".",
    # color="slategray",
    # density=True,
    alpha=0.5,
    # label=f"DAMIT ({len(damit):,d})",
)


fig.savefig(
    os.path.join("..", "gfx", "oblateness-scatter.png"), facecolor="white", dpi=150
)
# fig.savefig(os.path.join("..","gfx","oblateness-scatter.pgf"))

RuntimeError: The command
    xelatex -interaction=nonstopmode -halt-on-error figure.tex
failed and generated the following output:
This is XeTeX, Version 3.141592653-2.6-0.999993 (TeX Live 2022/dev/Debian) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./figure.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size12.clo))
(/usr/share/texlive/texmf-dist/tex/latex/hyperref/hyperref.sty
(/usr/share/texlive/texmf-dist/tex/generic/ltxcmds/ltxcmds.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)
(/usr/share/texlive/texmf-dist/tex/generic/pdftexcmds/pdftexcmds.sty
(/usr/share/texlive/texmf-dist/tex/generic/infwarerr/infwarerr.sty))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/kvsetkeys/kv

# R vs obliquity and range lambda

In [73]:
maskFINK = maskFINK_union

fig, ax = plt.subplots(
    1,
    2,
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "right": 0.98,
        "top": 0.95,
        "bottom": 0.15,
        "left": 0.1,
        "wspace": 0.02,
    },
    sharey=True,
)

# Histogram ranges and bins
r_R = [0.3, 1]
r_Lambda = [0, 90]
r_obliquity = [0, 90]

b_R = 35
b_Lambda = 36
b_obliquity = 36

vmin, vmax = 0, 200

# --------------------------------------------------------------------------------
# R vs Lambda
Lambda = np.degrees(
    np.arccos(data.loc[maskFINK, "SHG1G2_min_cos_lambda"])
    - np.arccos(data.loc[maskFINK, "SHG1G2_max_cos_lambda"])
)
_,_,_, im0 = ax[0].hist2d(
    Lambda,
    data.loc[maskFINK, "SHG1G2_R"],
    range=[r_Lambda, r_R],
    bins=[b_Lambda, b_R],
    cmap="Blues",
    norm="linear",
    rasterized=True,
    vmin=vmin, vmax=vmax,
    # label=f"DAMIT ({len(damit):,d})",
)


# --------------------------------------------------------------------------------
# Color bar
axinsr = inset_axes(
    ax[0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="lower right",
)
cbar_r = fig.colorbar(im0, cax=axinsr, orientation="horizontal")
cbar_r.ax.set_xticks([0,50,100,150,200])
cbar_r.ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
# cbar_r.ax.set_xticklabels(["1", "10", "100", "1000"], fontsize="small")



# --------------------------------------------------------------------------------
# R vs obliquity
obli = data.loc[maskFINK, "SHG1G2_obliquity"]
obli[obli > 90] = 180 - obli[obli > 90]

ax[1].hist2d(
    obli,
    data.loc[maskFINK, "SHG1G2_R"],
    range=[r_obliquity, r_R],
    bins=[b_obliquity, b_R],
    cmap="Blues",
    norm="linear",
    rasterized=True,
    # vmin=vmin, vmax=vmax,

)


# --------------------------------------------------------------------------------
# Axes
ax[0].set_ylabel("Oblateness R")
ax[0].set_xlabel(r"$\Delta \Lambda$ ($^{\circ}$)")
ax[1].set_xlabel(r"Obliquity ($^{\circ}$)")

ax[0].set_xticks(range(0, 91, 30))
ax[1].set_xticks(range(0, 91, 30))


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join("..", "gfx", "article", "oblateness-issues.png"),
    facecolor="white",
    dpi=150,
)
fig.savefig(os.path.join("..", "gfx", "article", "oblateness-issues.pgf"))

In [74]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)

In [75]:
all_bft = all.merge(bft, left_on="sso_name", right_on="sso_name", how="left")

In [76]:
fig, ax = plt.subplots()

vmin=0
vmax=500

r_H = [12,18]
b_H = 12

ax.hist2d( all_bft["SHG1G2_R"], all_bft["absolute_magnitude.value"],  rasterized=True,
          range=[r_R, r_H], bins=[b_R, b_H], cmap="Blues", norm="linear", vmin=vmin, vmax=vmax,)


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join("..", "gfx", "test-R.png"),
    facecolor="white",
    dpi=180,
)

/tmp/ipykernel_642503/1227003582.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
